In [1]:
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Flatten, Reshape, LeakyReLU
from tensorflow.keras.optimizers import Adam
import numpy as np
!mkdir gen_digit_images

In [2]:
img_width= 28
img_height= 28
channels= 1
img_shape= (img_width, img_height, channels)
latent_dim= 100
adam= Adam(learning_rate= 0.0001)

In [3]:
from keras.datasets import mnist

In [4]:
def build_generator():
    model = Sequential([
        Dense(256, activation='relu', input_dim= latent_dim),
        BatchNormalization(momentum= 0.8),
        Dense(512, activation='relu'),
        BatchNormalization(momentum= 0.8),
        Dense(1024,activation='relu'),
        BatchNormalization(momentum= 0.8),
        Dense(np.prod(img_shape), activation= 'tanh'),
        Reshape(img_shape)
    ])
    return model

generator= build_generator()

In [5]:
def build_discriminator():
    model = Sequential()
    model.add(Flatten(input_shape= img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation= 'sigmoid'))    
    return model

discriminator= build_discriminator()

In [6]:
discriminator.compile(optimizer= adam, loss= 'binary_crossentropy')
gan = Sequential()
gan.add(generator)
discriminator.trainable= False
gan.add(discriminator)

gan.compile(optimizer= adam, loss= 'binary_crossentropy')

In [7]:
import matplotlib.pyplot as plt
import glob
import imageio
import PIL

save_name= 0.00000000

def save_image(epoch):
    r, c = 5, 5 
    noise= np.random.normal(0,1, (r*c, latent_dim))
    gen_imgs= generator.predict(noise)
    global save_name
    save_name+= 0.00000001
    print("%.8f" % save_name)
    gen_imgs= 0.5 * gen_imgs + 0.5
    fig, axs= plt.subplots(r,c)
    cnt= 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt,:,:,0],cmap= 'gray')
            axs[i,j].axis('off')
            cnt+=1
    fig.savefig("gen_digit_images/%.8f.png" % save_name)
    plt.close()


In [8]:
def train(epochs, batch_size= 64, save_interval= 200):
    (x,_),(_,_)= mnist.load_data()
    x= x/ 127.5 -1.
    valid= np.ones((batch_size, 1))
    fake= np.zeros((batch_size, 1))
    
    for epoch in range(epochs):
        idx= np.random.randint(0, x.shape[0], batch_size)
        imgs= x[idx]
        
        noise= np.random.normal(0,1, (batch_size, latent_dim))
        gen_imgs= generator.predict(noise)
        
        # train discriminator
        d_loss_real= discriminator.train_on_batch(imgs, valid)
        d_loss_fake= discriminator.train_on_batch(gen_imgs, fake)
        d_loss= np.add(d_loss_real,d_loss_fake) * 0.5
        
        noise= np.random.normal(0, 1, (batch_size, latent_dim))
        g_loss= gan.train_on_batch(noise, valid)
        
        print(f'{epoch} [D loss {d_loss} ] [G loss {g_loss}]')
        
        if (epoch % save_interval) == 0:
            save_image(epoch)
        
train(30000)
        
        
        
    

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz: None -- [Errno 11001] getaddrinfo failed

In [12]:
anim_file = 'dcgan.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('gen_digit_images/*.png')
  filenames = sorted(filenames)
  for filename in filenames:
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)